In [2]:
import requests
import time, json, re
from pprint import pprint
import pandas as pd
from yap_tools import *

In [3]:
df = pd.read_pickle(r"C:\\Users\\Alex\\Documents\\datasets\\halalim\\halalim2.pkl")

In [4]:
df["has_life_story"]

0        True
1        True
2        True
3        True
4        True
         ... 
23655    True
23656    True
23657    True
23658    True
23659    True
Name: has_life_story, Length: 23660, dtype: bool

In [5]:
df.head(1).T

,0
age,23
cemetery_code,720
cemetery_hierarchy,None
cemetery_name,הר הזיתים
cemetery_type,2
day_of_commemoration,NaN
death_date,5-1-1873
death_date_day_and_month_hebrew,טבת ו
death_date_hebrew,"ו' בטבת תרל""ג"
death_date_month,1


In [6]:
bpdf = df[df["has_life_story"]]["life_story"].str.split(r"\.\s+", n=4, expand=True).fillna(value='')

In [7]:
df[df["has_life_story"]][["life_story", "uuid"]].to_csv("halalim.csv", encoding="utf-8-sig")

In [8]:
df[df["has_life_story"]]["life_story"].loc[23659]

'בן ניצן ופדרו. נולד ביום ג\' בכסלו תשנ"ט (22.11.1998) בנתניה. ילד שני במשפחה, אח של קורל ושובל. ביום 2.5.2018 התגייס שחר, תושב נתניה, לצה"ל והחל טירונות. טוראי שחר רוּבילָר נפל בעת שירותו ביום י"ד בסיוון תשע"ח (28.5.2018). בן עשרים בנופלו. הוא הובא למנוחות בבית העלמין הצבאי בנתניה. הותיר הורים ושתי אחיות. על מצבתו כתבו אוהביו: "מטרת הקיים היא חיים עם מטרה. יום ללא חיוך הוא יום מבוזבז"'

In [9]:
def ccount(s): return re.subn(r'נולד', '', s)[1]

scores = bpdf.applymap(ccount).astype(bool)

In [ ]:
l = []
for index, row in bpdf.iterrows():
    for column, content in row.iteritems():
        if scores.loc[index, column]:
            time.sleep(3)
            content = skip_dates(content)
            print(content)
            d = yap_it(content)
            l += [v["word"] for v in d["md_lattice"].values() if v["pos"] == "NNP"]
            with open(f"yap{index}.json", "w", encoding="utf-8") as fp:
                json.dump(d, fp, ensure_ascii=False)
    if index > 10:
        break
l = [x for x in l if len(x) > 1]

נולד ב (1850) בהונגריה
נולד ב (1846) בשכונת בתי מחסה שבעיר העתיקה בירושלים
נולד ב (1850) בביאליסטוק, פולין
נולד ב (1864), בעיר קיוב שברוסיה למשפחת מוגילביץ ברוכת הילדים כשהגיע לגיל שירות צבאי אומץ על ידי דודו חשוך הבנים כדי לפטרו מן השרות כבן יחיד
נולד בבולגריה ועלה לארץ-ישראל בתקופת העליה הראשונה


In [12]:
def skip_dates(s):
    months = ["תשרי", "חשון", "חשוון", "מרחשוון", "כסלו", "טבת", "שבט", "אדר", "ניסן", "אייר", "איר", "סיוון", "סיון", "תמוז", "אב", "אלול"]
    s = re.sub(r"שנת \w{2,3}\"\w", "", s)
    mylist = s.split()
    r = re.compile(f".*({'|'.join(months)}).*")
    for x in list(filter(r.match, mylist)):
        idx = mylist.index(x)-1
        if "'" in mylist[idx] or "\"" in mylist[idx]:
            for _ in range(3):
                mylist.pop(idx)
    return " ".join(mylist)

In [ ]:
with open("yap4.json", "r", encoding="utf-8") as f:
    data = json.load(f)

longest life_story 14688, id: 23279

In [ ]:
fmted = fmt_dep_tree(data["dep_tree"])

In [ ]:
data["tokenized_text"]

In [248]:
lattice

,empty,gen,lemma,num,num_2,num_last,num_s_p,per,pos,pos_2,tense,word
0,-1,M,נולד,0,1,1,S,-1,NNT,NNT,-1,נולד
1,-1,M,ב,1,2,2,S,-1,NNP,NNP,-1,ב
2,-1,-1,_,2,3,3,-1,-1,CD,CD,-1,1864
3,-1,-1,_,3,4,4,-1,-1,yyCM,yyCM,-1,","
4,-1,-1,ב,4,5,5,-1,-1,PREPOSITION,PREPOSITION,-1,ב
5,-1,-1,ה,5,6,5,-1,-1,DEF,DEF,-1,ה
6,-1,F,עיר,6,7,5,S,-1,NN,NN,-1,עיר
7,-1,F,קיוב,7,8,6,S,-1,NNP,NNP,-1,קיוב
8,-1,-1,ש,8,9,7,-1,-1,REL,REL,-1,ש
9,-1,-1,ב,9,10,7,-1,-1,PREPOSITION,PREPOSITION,-1,ב


* gen = gender
* lemma = base word repr

* __NNP__ = proper noun 
* __NNT__ = ?
* __NN__ = noun

In [236]:
data["dep_tree"]

{'0': {'num': '1',
  'word': 'נולד',
  'lemma': 'נולד',
  'pos': 'NNT',
  'pos_2': 'NNT',
  'empty': 'gen=M|num=S',
  'dependency_arc': '25',
  'dependency_part': 'subj',
  'dependency_arc_2': '_',
  'dependency_part_2': '_'},
 '1': {'num': '2',
  'word': 'ב',
  'lemma': 'ב',
  'pos': 'NNP',
  'pos_2': 'NNP',
  'empty': 'gen=F|gen=M|num=S',
  'dependency_arc': '1',
  'dependency_part': 'gobj',
  'dependency_arc_2': '_',
  'dependency_part_2': '_'},
 '2': {'num': '3',
  'word': '1864',
  'lemma': '1864',
  'pos': 'CD',
  'pos_2': 'CD',
  'empty': '',
  'dependency_arc': '2',
  'dependency_part': 'nn',
  'dependency_arc_2': '_',
  'dependency_part_2': '_'},
 '3': {'num': '4',
  'word': ',',
  'lemma': ',',
  'pos': 'yyCM',
  'pos_2': 'yyCM',
  'empty': '',
  'dependency_arc': '1',
  'dependency_part': 'punct',
  'dependency_arc_2': '_',
  'dependency_part_2': '_'},
 '4': {'num': '5',
  'word': 'ב',
  'lemma': 'ב',
  'pos': 'PREPOSITION',
  'pos_2': 'PREPOSITION',
  'empty': '',
  'depend

In [244]:
patterns = pd.DataFrame(columns=lattice.columns)

In [246]:
patterns = patterns.append(lattice.loc[4:7])

In [256]:
def count_patterns(lattice, word, begin, end):
    pat = " ".join(lattice["pos"].loc[begin:end].tolist())
    if count.get(pat, False):
        count[pat] += 1
    else:
        count[pat] = 1

In [255]:
count = {}

In [1]:
displacy.render(fmted, style="dep", manual=True, page=False, minify=True)

NameError: name 'displacy' is not defined

In [184]:
a

gf root - root
hd - head (governor, argument-taking)
prd - verbal predicate
exist - head of an existential phrase
nhd - head of a nominal phrase
ghd - genitive head of a nominal phrase
dep - dependent (governed, or an argument)
arg - argument
agent - agent
comp - complement
acomp - adjectival complement
ccomp - comp clause with internal sbj
xcomp - comp clause with external sbj
pcomp - comp clause of a preposition
obj - object
dobj - direct object
gobj - genitive object
iobj - indirect object
pobj - object of a preposition
subj - subject
expl - expletive subject
nsubj - nominal subject
— nsubjpass - passive nominal sbj
csubj - clausal subject
— csubjpass - passive clausal sbj
mod - modifier
appos - apposition/parenthetical
abbrev - abbreviation
amod - adjectival modifier
advmod - adverbial modifier
— neg - negative modifier
prepmod - prepositional modifier
— possmod - possession modifier
— tmod - temporal modifier
rcmod - relative clause modifier
infmod - infinitival modifier
nummod - numerical modifier
parataxis - ”side-by-side”, interjection
conj - conjuct
func - functional (argument marking)
marker - nominal-marking elements
prep - preposition
case - case marker
— acc - accusative case
— dat - dative case
— gen - genitive case
— nom - nominative case
det - determiner
— def - definite marker
— dem - demonstrative
sub - phrase-marking elements
complm - introducing comp phrase
rel - introducing relative phrase
cc - introducing conjunction
mark - introducing an advb phrase
aux - auxiliary verb or a feature-bundle
auxpass - passive auxiliary
cop - copular element
modal - modal verb
qaux - question auxiliary
punct - punctuation